In [ ]:
import pandas as pd
import numpy as np

## EDA Problem 1
Meteorologia

Generacion_fotovoltaica

Consumo_fotovoltaica

In [43]:
meteo = pd.read_csv("datasets/Meteorologia.csv", sep=",")
genfot = pd.read_csv("datasets/Generacion_fotovoltaica.csv", sep=",")
confot = pd.read_csv("datasets/Consumo_fotovoltaica.csv", sep=",")